# CREATE RAG

### Установка библиотек

In [1]:
!pip install -q llama-index llama-index-readers-web firecrawl-py
!pip install -q tokenizers
!pip install -q llama-index-llms-openrouter
!pip install -q "llama-index-core>=0.10.43" "openinference-instrumentation-llama-index>=2" "opentelemetry-proto>=1.12.0" arize-phoenix arize-phoenix-otel
!pip install -q llama-index llama-hub
!pip install -q llama-index-embeddings-huggingface
!pip install -q llama-index-embeddings-langchain
!pip install -q llama-index-embeddings-together
!pip install -q langchain-huggingface
!pip install -q gcsfs nest-asyncio
!pip install -q requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 54.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

### API_KEYS и константы

In [2]:
from google.colab import userdata
from huggingface_hub import login


# openrouter
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
OPENROUTER_LLM_MODEL_NAME = "meta-llama/llama-3.2-3b-instruct:free"

# together.ai embedding
TOGETHER_API_KEY = userdata.get('TOGETHER_API_KEY')
embed_models_together_ai = [  # все модели для эмбединга из документации
    "togethercomputer/m2-bert-80M-2k-retrieval",
    "togethercomputer/m2-bert-80M-8k-retrieval",
    "togethercomputer/m2-bert-80M-32k-retrieval",
    "WhereIsAI/UAE-Large-V1",
    "BAAI/bge-large-en-v1.5",
    "BAAI/bge-base-en-v1.5",
    "sentence-transformers/msmarco-bert-base-dot-v5",
    "bert-base-uncased",
]
TOGETHER_AI_EMBED_MODEL = embed_models_together_ai[1]

# local embedding
EMBED_MODEL_NAME = "BAAI/bge-m3"  # multilingual
# EMBED_MODEL_NAME = "BAAI/bge-large-en-v1.5"  # only english

# tokenizer
DEFAULT_TOKENIZER_NAME = 'llama-3.2'

# parsing
FIRE_CRAWL_API_KEY = userdata.get('FIRE_CRAWL_API_KEY')

# hf
login(userdata.get("HF_TOKEN"))


## Парсер

In [3]:
from llama_index.readers.web import FireCrawlWebReader
from pathlib import Path
from typing import List

import os


firecrawl_reader = FireCrawlWebReader(
    api_key=FIRE_CRAWL_API_KEY,
    mode="scrape",
)
FOLDER_FOR_PARSED_TEXT = 'parsed_text'



def read_documents_from_url(url: str) -> List[str]:
    documents = firecrawl_reader.load_data(url=url)
    texts = []
    if isinstance(documents, list):
        for doc in documents:
            if hasattr(doc, 'text'):
                texts.append(doc.text)
            else:
                print("В документе не найден атрибут «текст».")
    else:
        print("Неожиданный формат документов:", documents)
    return texts


def get_dir_path(filename: str) -> Path:
    return Path(FOLDER_FOR_PARSED_TEXT, filename)


def get_filepath(filename: str) -> Path:
    dir = get_dir_path(filename)
    os.makedirs(dir, exist_ok=True)
    return Path(dir, f'{filename}.txt')


def save_parsed_text_to_file(
    filename: str,
    texts: List[str],
) -> None:
    file_path = get_filepath(filename)
    with open(file_path, 'w', encoding='utf-8') as file:
        for text in texts:
            file.write(text + '\n')


def load_parsed_text(filename: str) -> str:
    file_path = get_filepath(filename)
    with open(file_path, 'r', encoding='utf-8') as f:
        file_content = f.read()
    return file_content


/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "schema" in "FirecrawlApp.ExtractParams" shadows an attribute in parent "BaseModel"
  warnings.warn(


## Токенизатор

In [4]:
from tokenizers import Tokenizer


downloaded_tokenizers = {
    "llama-3.1": Tokenizer.from_pretrained("Xenova/Meta-Llama-3.1-Tokenizer"),
    "llama-3.2": Tokenizer.from_pretrained("pcuenq/Llama-3.2-1B-Instruct-tokenizer"),
    "zephyr": Tokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta"),
    "grok-1": Tokenizer.from_pretrained("Xenova/grok-1-tokenizer"),
    "gpt-4": Tokenizer.from_pretrained("Xenova/gpt-4"),
    "gpt-4o": Tokenizer.from_pretrained("Xenova/gpt-4o"),
    "gpt-3.5-turbo": Tokenizer.from_pretrained("Xenova/gpt-3.5-turbo"),
    "deepseek-2.5": Tokenizer.from_pretrained("deepseek-ai/DeepSeek-V2.5"),
}
DEFAULT_TOKENIZER = downloaded_tokenizers.get(DEFAULT_TOKENIZER_NAME, "llama-3.1")


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.23M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.73M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.23M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

In [5]:
from tokenizers import Tokenizer
from typing import Dict


def encode_text(
    text: str,
    tokenizer: Tokenizer = DEFAULT_TOKENIZER,
) -> Dict:
    encoded = tokenizer.encode(text)
    ids, tokens = encoded.ids, encoded.tokens
    return {
        "ids": ids,
        "tokens": tokens,
    }


def count_tokens(
    text: str,
    tokenizer: Tokenizer = DEFAULT_TOKENIZER,
) -> int:
    encoded = encode_text(text, tokenizer=tokenizer)
    return len(encoded["ids"])


## RAG

### Трасировка

In [6]:
import nest_asyncio
import phoenix as px


nest_asyncio.apply()
session = px.launch_app()

🌍 To view the Phoenix app in your browser, visit https://94gnjockm41-496ff2e9c6d22116-6006-colab.googleusercontent.com/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [7]:
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor


endpoint = "http://127.0.0.1:6006/v1/traces"

tracer_provider = TracerProvider()
tracer_provider.add_span_processor(
    SimpleSpanProcessor(OTLPSpanExporter(endpoint))
)

LlamaIndexInstrumentor().instrument(
    skip_dep_check=True,
    tracer_provider=tracer_provider
)

### Промпт

In [8]:
SYSTEM_PROMPT = """
Ты дружелюбный помощник отвечающий на вопросы. Отвечай основываясь на контексте.
Отвечай максимально подробно и обстоятельно. Не выдумывай факты.
"""
TEXT_START = "<|begin_of_text|>"
START_HEADER = "<|start_header_id|>"
END_HEADER = "<|end_header_id|>"
EOT_ID = "<|eot_id|>"


def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"{TEXT_START}{START_HEADER}{message.role}{END_HEADER}\n{SYSTEM_PROMPT}"
        elif message.role == 'user':
            prompt += f"{EOT_ID}{START_HEADER}{message.role}{END_HEADER}\n{message.content}"
        elif message.role == 'assistant':
            prompt += f"{EOT_ID}{START_HEADER}{message.role}{END_HEADER}\n"

    if not prompt.startswith(TEXT_START):
        prompt = f"{TEXT_START}{START_HEADER}{message.role}{END_HEADER}\n{SYSTEM_PROMPT}" + prompt

    return prompt

def completion_to_prompt(completion):
    prompt = f"{START_HEADER}system{END_HEADER}\n"\
             f"{SYSTEM_PROMPT}{EOT_ID}{START_HEADER}user{END_HEADER}\n"\
             f"{completion}{EOT_ID}{START_HEADER}assistant{END_HEADER}\n"
    return prompt


### LLM

In [9]:
from llama_index.llms.openrouter import OpenRouter
from google.colab import userdata


MAX_TOKENS = 4096
MAX_CONTEXT = 8192
TEMPERATURE = 0.1

model_settings = {
    "api_key": OPENROUTER_API_KEY,
    "model": OPENROUTER_LLM_MODEL_NAME,
    "max_tokens": MAX_TOKENS,
    "context_window": MAX_CONTEXT,
    "temperature": TEMPERATURE,
    "system_prompt": SYSTEM_PROMPT,
    "messages_to_prompt": messages_to_prompt,
    "completion_to_prompt": completion_to_prompt,
}


llm = OpenRouter(**model_settings)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

### Модель для эмбединга

#### Локальный

In [10]:
from langchain_huggingface  import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding


local_embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name=EMBED_MODEL_NAME)
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

#### Together.AI [embedding- models](https://docs.together.ai/docs/serverless-models#embedding-models)

In [11]:
from llama_index.embeddings.together import TogetherEmbedding


together_ai_embed_model = TogetherEmbedding(
    model_name=TOGETHER_AI_EMBED_MODEL,
    api_key=TOGETHER_API_KEY
)


### Глобальные настройки для RAG

In [12]:
from llama_index.core import Settings


CHUNK_SIZE = 512
CHUNK_OVERLAP = 64
CHUNK_SIZES = [2048, 1024, 256]


Settings.llm = llm
Settings.chunk_size = CHUNK_SIZE
Settings.embed_model = together_ai_embed_model
# Settings.embed_model = local_embed_model


### Создаем индексированные данные для RAG (AutoMergingRetriever)

In [13]:
from llama_index.core import SimpleDirectoryReader, Document
from llama_index.core.node_parser import HierarchicalNodeParser
from llama_index.core.node_parser import get_leaf_nodes


PATH_FOLDER_INDEX = 'indexes'


def get_path_index(key: str) -> Path:
    return Path(PATH_FOLDER_INDEX, key)


def get_documents(key: str) -> List[Document]:
    documents = SimpleDirectoryReader(
        get_dir_path(key)
    ).load_data()
    return documents


def get_nodes(key: str) -> tuple:
    documents = get_documents(key)
    node_parser = HierarchicalNodeParser.from_defaults(
        chunk_sizes=CHUNK_SIZES,
        chunk_overlap=CHUNK_OVERLAP,
    )
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)
    return nodes, leaf_nodes


In [14]:
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext


def parse_url(name: str, url: str) -> None:
    # определяем путь к сохранению индексированных данных
    path_index = get_path_index(name)
    os.makedirs(path_index, exist_ok=True)
    # парсим документацию по ссылке
    texts = read_documents_from_url(url)
    # сохраняем спарсенные данные в файл
    save_parsed_text_to_file(name, texts)
    print(f"Данные успешно спарсены по url: {url}")
    # подсчет коливества токенов в контексте
    n_tokens = count_tokens(load_parsed_text(name))
    print(f"Количество токенов в контексте: {n_tokens}")


def parse_and_read_data(name: str, url: str) -> List[Document]:
    parse_url(name, url)
    return get_documents(name)


def create_vector_store_index_files(name: str, url: str) -> None:
    # определяем путь к сохранению индексированных данных
    path_index = get_path_index(name)
    os.makedirs(path_index, exist_ok=True)
    # парсим документацию по ссылке
    parse_url(name, url)
    # получаем узлы из сохраненного файла с помощью HierarchicalNodeParser
    nodes, leaf_nodes = get_nodes(name)
    # организуем хранилище
    docstore = SimpleDocumentStore()
    docstore.add_documents(nodes)
    storage_context = StorageContext.from_defaults(docstore=docstore)
    # индексируем данные
    index = VectorStoreIndex(
        leaf_nodes,
        storage_context=storage_context,
        embed_model=Settings.embed_model
    )
    index.set_index_id(name)
    # сохраняем индексированные данные по указанному пути
    index.storage_context.persist(path_index)
    print(f"Данные успешно проиндексированы и сохранены ({path_index})")


### Создание QueryEngine из спарсенных Document's с помощью AutoMergingRetrieverPack

In [15]:
from llama_index.core.llama_pack import download_llama_pack


def get_query_engine_from_documents(documents):
    AutoMergingRetrieverPack = download_llama_pack(
        "AutoMergingRetrieverPack",
        "./auto_merging_retriever_pack",
    )
    auto_merging_retriver = AutoMergingRetrieverPack(
        documents,
        chunk_sizes=CHUNK_SIZES,
        chunk_overlap=CHUNK_OVERLAP,
    )
    print("QueryEngine (AutoMergingRetriever) успешно создан!")
    return auto_merging_retriver


def create_RAG_AutoMergingRetrieverPack(name: str, url: str):
    """Usage: query_engine.run('some_query_str')"""
    return get_query_engine_from_documents(
        parse_and_read_data(name, url)
    )


### Создание QueryEngine из сохраненных хранилищ индексированных данных

In [16]:
from llama_index.core import (
    load_index_from_storage,
    StorageContext,
)
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine


def get_query_engines_from_storage(key: str, top_k: int):
    # пересобираем хранилище
    storage_context = StorageContext.from_defaults(
        persist_dir=get_path_index(key)
    )
    # загружаем индексированные данные
    index_loaded = load_index_from_storage(
        storage_context, index_id=key, embed_model=Settings.embed_model
    )
    print("Проиндексированные данные успешно загружены")
    # создаем базовый поисковик
    base_retriever = index_loaded.as_retriever(
        similarity_top_k=top_k
    )
    # создаем AutoMergingRetriever на основе базового
    retriever = AutoMergingRetriever(
        base_retriever, storage_context, verbose=True
    )
    # создаем движки запросов
    query_engine = RetrieverQueryEngine.from_args(base_retriever)
    query_engine_amr = RetrieverQueryEngine.from_args(retriever)

    print("Успешно созданы 2 query_engine (base_retriver, auto_merging_retriver)")

    return query_engine, query_engine_amr


def create_RAG_from_storage_content(
    name: str,
    url: str,
    top_k: int = 6,
) -> tuple[RetrieverQueryEngine, RetrieverQueryEngine]:
    create_vector_store_index_files(name, url)
    return get_query_engines_from_storage(name, top_k)


## Пользовательский RAG

##### AutoMergingRetrieverPack

In [19]:
%%time
name = 'alternate_syndroms'  # произвольное имя
url = 'https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D1%8C%D1%82%D0%B5%D1%80%D0%BD%D0%B8%D1%80%D1%83%D1%8E%D1%89%D0%B8%D0%B5_%D1%81%D0%B8%D0%BD%D0%B4%D1%80%D0%BE%D0%BC%D1%8B'


query_engine_pack = create_RAG_AutoMergingRetrieverPack(name, url)

Данные успешно спарсены по url: https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D1%8C%D1%82%D0%B5%D1%80%D0%BD%D0%B8%D1%80%D1%83%D1%8E%D1%89%D0%B8%D0%B5_%D1%81%D0%B8%D0%BD%D0%B4%D1%80%D0%BE%D0%BC%D1%8B
Количество токенов в контексте: 10652
QueryEngine (AutoMergingRetriever) успешно создан!
CPU times: user 6.34 s, sys: 599 ms, total: 6.94 s
Wall time: 1min 26s


In [21]:
prompt = """
You are a highly qualified neurologist's assistant who advises on
various medical issues. Answer in as much detail as possible, based on
the context. Don't make up the answers. If you don't know the answer,
be honest about it.
"""
Settings.llm.system_prompt = prompt

query = "какая клиника у синдрома Мийяра—Гублера, ответь максимально развернуто"
response = query_engine_pack.run(query)
print(response)

Синдром Мийяра—Гублера — это неврологический синдром, характеризующийся параличом мышц лица и языка, а также гиперрефлексиями в других частях тела. 

Клиника синдрома Мийяра—Гублера может включать следующие симптомы:

1. **Паралич мышц лица**: Сильное уменьшение или полное паралич мышц лица, что приводит к выражению скудности и отсутствия эмоциональных выражений.
2. **Паралич мышц языка**: Сильное уменьшение или полное паралич мышц языка, что приводит к трудностям сarticulation и пониманием речи.
3. **Гемиплегия**: Сильное уменьшение или полное паралич одной стороны тела, что может включать мышцы конечностей, спины и трапециевидной мышцы.
4. **Гиперрефлексия**: Увеличение рефлексов в других частях тела, что может включать мышцы конечностей, спины и трапециевидной мышцы.
5. **Нарушения координации и баланса**: Трудности с координацией и балансом, что может привести к падениям или несчастным случаям.
6. **Торчевание**: Торчевание или сжатие мышц, что может привести к дискомфорте и болью.

In [27]:
query_2 = "Ответь максимально подробно. Какие синдромы называют альтернирующими? Чем они заслужили такое название?"

response_2 = query_engine_pack.run(query_2)
print(response_2)

Альтернирующие синдромы - это группа неврологических расстройств, характеризующихся альтернирующим или чередующимся признаками, которые могут включать в себя различную комбинацию симптомов, таких как головные боли, тошнота, дизентерия, астения, и т. д. Эти синдромы часто не имеют четкой и устойчивой этиологии и могут быть вызваны различными факторами, включая генетические, инфекционные, токсические и другие факторы.

Альтернирующие синдромы получили такое название из-за того, что их симптомы чередуются или альтернируют друг с другом, что может создать трудность в диагностике и лечении. Эти синдромы часто могут быть confuse с другими неврологическими расстройствами или даже с психическими расстройствами.

Некоторые из наиболее известных альтернирующих синдромов включают:

1. **Грудино-ключично-сосцевидная мышечная дистония**: характеризуется альтернирующим набором симптомов, включая мышечную тENSENCE в грудино-ключично-сосцевидной мышце, тошноту, дизентерия и астению.
2. **Кровоизлияния

##### RAG from storage content

In [28]:
%%time
_, query_engine = create_RAG_from_storage_content(name, url)


Данные успешно спарсены по url: https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D1%8C%D1%82%D0%B5%D1%80%D0%BD%D0%B8%D1%80%D1%83%D1%8E%D1%89%D0%B8%D0%B5_%D1%81%D0%B8%D0%BD%D0%B4%D1%80%D0%BE%D0%BC%D1%8B
Количество токенов в контексте: 10652
Данные успешно проиндексированы и сохранены (indexes/alternate_syndroms)
Проиндексированные данные успешно загружены
Успешно созданы 2 query_engine (base_retriver, auto_merging_retriver)
CPU times: user 4.01 s, sys: 282 ms, total: 4.29 s
Wall time: 37 s


In [29]:
response = query_engine.query(query)
print(response)

Синдром Мийяра—Гублера является альтернирующим синдромом, который возникает при поражении одной половины ствола головного мозга, спинного мозга. Этот синдром характеризуется различными клиническими признаками, которые зависят от конкретного очага поражения и его границ.

Клиника синдрома Мийяра—Гублера может включать в себя следующие признаки:

1. **Паралич**: Поражение ядер и корешков спинного мозга может привести к параличам различных частей тела, включая конечности, тазовую область и тело.
2. **Неврологические симптомы**: Поражение корешков спинного мозга может привести к неврологическим симптомам, таким как дискомпенсация, гиперрефлексия, гиперсестезия и диффузный зуд.
3. **Патологические изменения в спинном мозге**: Поражение спинного мозга может привести к патологическим изменениям в спинном мозге, включаяatroфию и сцилероз.
4. **Патологические изменения в теле**: Поражение ядер и корешков спинного мозга может привести к патологическим изменениям в теле, включаяatroфию и сцилероз

In [30]:
response_2 = query_engine.query(query_2)
print(response_2)

> Merging 1 nodes into parent node.
> Parent node id: 01876efd-675e-4c55-8362-b7c6c8834138.
> Parent node text: ](/wiki/%D0%9B%D0%B0%D1%82%D0%B8%D0%BD%D1%81%D0%BA%D0%B8%D0%B9_%D1%8F%D0%B7%D1%8B%D0%BA "Латински...

Альтернирующие синдромы — это группа синдромов, которые характеризуются чередованием поражения черепных нервов на стороне очага с проводниковыми расстройствами двигательной и чувствительной функций на противоположной стороне. Этот синдром возникает при поражении черепных нервов на стороне очага, что приводит к нарушению функций на стороне, противоположной очага, и vice versa.

Альтернирующие синдромы называются так из-за того, что функции на двух сторонах черепа чередуются, как в альтернативном порядке. Это означает, что при поражении черепных нервов на одной стороне функции на противоположной стороне могут быть нарушены, а затем, когда функции на первой стороне возвращаются, функции на второй стороне могут быть нарушены.

Этот синдром заслужил такое название из-за того, что 

##### Выводы

- при использовании самостоятельного создания StorageContext - RAG гораздо меньше галлюцинирует и более полно опирается на контекст
- при использовании AutoMergingRetieverPack - по сути RAG не ответил ни на один из заданных вопросов и выдавал только странные общие фразы и галлюцинации
- при создании RAG на основе StorageContext - RAG ответил правильно на оба поставленных вопроса. На первый запрос - тоже присутствуют галлюцинации но в меньшем количестве. На второй вопрос - ответ очень даже хороший, точный и полный.
- есть необходимость подобрать русскоязычные llm и embed_model для улучшения качества ответов и уменьшения рунглиша :)